# Week 2: Generation with MC Dropout Uncertainty

This notebook demos the generation pipeline built in Week 2:
- **MCDropoutModel** — wraps distilgpt2 with stochastic forward passes
- **UncertaintyEstimator** — predictive entropy, mutual information, diversity
- **UncertainGenerator** — ties it all together with retrieval results

We'll walk through each piece, then run the full pipeline end-to-end.

In [1]:
import sys
sys.path.insert(0, '..')

import torch
import numpy as np
from src.generation import MCDropoutModel, UncertaintyEstimator, UncertainGenerator

## 1. MC Dropout Model

Load distilgpt2 and verify dropout layers exist.

In [2]:
model = MCDropoutModel(model_name='distilgpt2')

import torch.nn as nn
dropout_layers = [m for m in model.model.modules() if isinstance(m, nn.Dropout)]
print(f'Found {len(dropout_layers)} dropout layers in distilgpt2')
print(f'Device: {model.device}')

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Found 19 dropout layers in distilgpt2
Device: cpu


### Dropout toggle demo

Show that enabling dropout puts only those layers in train mode.

In [3]:
model._enable_dropout()
for m in model.model.modules():
    if isinstance(m, nn.Dropout):
        print(f'Dropout training={m.training}, p={m.p}')
        break

model._disable_dropout()
print('\nAfter disable:')
for m in model.model.modules():
    if isinstance(m, nn.Dropout):
        print(f'Dropout training={m.training}')
        break

Dropout training=True, p=0.1

After disable:
Dropout training=False


### Generate multiple samples

With dropout on, each forward pass uses a different mask — so we get diverse outputs.

In [4]:
prompt = "The capital of France is"
result = model.generate(prompt, n_samples=5, max_new_tokens=30)

print(f'Prompt: "{prompt}"\n')
for i, txt in enumerate(result['texts']):
    print(f'Sample {i+1}: {txt[:80]}...' if len(txt) > 80 else f'Sample {i+1}: {txt}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: "The capital of France is"

Sample 1:  the nation of the European Empire. The region of the European countries of the ...
Sample 2:  a city where the public's bodies and the public live. But the public's rights c...
Sample 3:  in the midst of the current economic crisis. This year's economic and social pr...
Sample 4:  in French and British eyes. If you are lucky it could be the largest private ai...
Sample 5:  the second highest ever capital in France. In April, 2012, there were 18,000 fo...


### Forward pass logits

Get raw logits for uncertainty computation. Shape: `(n_samples, seq_len, vocab_size)`

In [5]:
input_ids = model.tokenizer(prompt, return_tensors='pt')['input_ids']
logits = model.forward_pass(input_ids, n_samples=10)
print(f'Logits shape: {logits.shape}')
print(f'  -> {logits.shape[0]} MC samples, {logits.shape[1]} tokens, {logits.shape[2]} vocab size')

Logits shape: torch.Size([10, 5, 50257])
  -> 10 MC samples, 5 tokens, 50257 vocab size


## 2. Uncertainty Estimation

The core math: predictive entropy (total), expected entropy (aleatoric), and mutual information (epistemic).

In [6]:
estimator = UncertaintyEstimator()

h_pred = estimator.predictive_entropy(logits)
h_exp = estimator.expected_entropy(logits)
mi = estimator.mutual_information(logits)

print('Per-token uncertainty metrics:')
print(f'  Predictive entropy (total):    {h_pred.numpy().round(3)}')
print(f'  Expected entropy (aleatoric):  {h_exp.numpy().round(3)}')
print(f'  Mutual information (epistemic): {mi.numpy().round(3)}')
print(f'\nSequence-level:')
print(f'  Mean entropy: {estimator.sequence_entropy(logits):.4f}')
print(f'  Mean MI:      {estimator.sequence_mi(logits):.4f}')

Per-token uncertainty metrics:
  Predictive entropy (total):    [7.857 3.138 5.887 3.238 5.489]
  Expected entropy (aleatoric):  [7.781 3.015 5.772 3.    5.389]
  Mutual information (epistemic): [0.075 0.123 0.116 0.238 0.1  ]

Sequence-level:
  Mean entropy: 5.1218
  Mean MI:      0.1303


### Sanity check: MI should always be between 0 and H_predictive

In [7]:
assert (mi >= 0).all(), 'MI should be non-negative'
assert (mi <= h_pred + 1e-5).all(), 'MI should be <= predictive entropy'
print('Checks passed: 0 <= MI <= H_predictive for all tokens')

Checks passed: 0 <= MI <= H_predictive for all tokens


### Text diversity metrics

In [8]:
samples = result['texts']
print(f'Unique ratio: {estimator.unique_ratio(samples):.2f}')
print(f'Pairwise diversity: {estimator.pairwise_diversity(samples):.3f}')

Unique ratio: 1.00
Pairwise diversity: 0.932


### Compare: high vs low uncertainty inputs

Uniform logits (max uncertainty) vs peaked logits (min uncertainty).

In [9]:
vocab = 50

# uniform -> high entropy
uniform_logits = torch.zeros(5, 3, vocab)
h_uniform = estimator.sequence_entropy(uniform_logits)

# peaked -> low entropy
peaked_logits = torch.full((5, 3, vocab), -100.0)
peaked_logits[:, :, 0] = 100.0
h_peaked = estimator.sequence_entropy(peaked_logits)

print(f'Uniform logits entropy:  {h_uniform:.4f}  (max possible: {np.log(vocab):.4f})')
print(f'Peaked logits entropy:   {h_peaked:.6f}  (near zero)')

Uniform logits entropy:  3.9120  (max possible: 3.9120)
Peaked logits entropy:   0.000000  (near zero)


## 3. Full Pipeline: UncertainGenerator

Simulate retrieval docs (as if coming from Week 1's BayesianRetriever) and run the full generation pipeline.

In [10]:
# fake retrieval results with posterior uncertainty from Week 1
fake_docs = [
    {
        'document': 'The Eiffel Tower is a wrought-iron lattice tower in Paris, France. '
                    'It was constructed from 1887 to 1889 as the centerpiece of the 1889 World\'s Fair.',
        'posterior_mean': 0.72,
        'posterior_std': 0.11,
        'entropy': 0.45,
        'ci_lower': 0.52,
        'ci_upper': 0.88,
    },
    {
        'document': 'Paris has a population of approximately 2.1 million residents. '
                    'The city is a major European center for finance, diplomacy, and the arts.',
        'posterior_mean': 0.65,
        'posterior_std': 0.09,
        'entropy': 0.38,
        'ci_lower': 0.48,
        'ci_upper': 0.81,
    },
]

print('Simulated retrieval docs:')
for i, d in enumerate(fake_docs):
    print(f'  [{i+1}] posterior_mean={d["posterior_mean"]}, std={d["posterior_std"]}, entropy={d["entropy"]}')

Simulated retrieval docs:
  [1] posterior_mean=0.72, std=0.11, entropy=0.45
  [2] posterior_mean=0.65, std=0.09, entropy=0.38


In [11]:
generator = UncertainGenerator(
    mc_model=model,
    estimator=estimator,
    default_samples=5,
    max_new_tokens=40,
)

result = generator.generate('Where is the Eiffel Tower located?', fake_docs)

print('=== Generated Prompt ===')
print(result['prompt'])
print()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== Generated Prompt ===
Context:
[1] The Eiffel Tower is a wrought-iron lattice tower in Paris, France. It was constructed from 1887 to 1889 as the centerpiece of the 1889 World's Fair.
[2] Paris has a population of approximately 2.1 million residents. The city is a major European center for finance, diplomacy, and the arts.

Question: Where is the Eiffel Tower located?
Answer:



In [12]:
print('=== MC Dropout Samples ===')
for i, s in enumerate(result['samples']):
    print(f'  [{i+1}] {s[:100]}' + ('...' if len(s) > 100 else ''))
print()

=== MC Dropout Samples ===
  [1]  What is the Eiffel Tower?
Question: The Eiffel Tower was originally built in 1890, and was construc...
  [2]  According to estimates of U.S. visitors in each of the six cities with the Eiffel Tower, up to 200 ...
  [3]  There is a building called the Eiffel Tower in Paris on a five-story structure and an underground t...
  [4]  It is a building of the world's biggest trade center.
Question: Why are these towers located inside...
  [5]  The Eiffel Tower is a substantial structure, constructed from 1904, in the city and used in the con...



In [13]:
print('=== Generation Uncertainty ===')
unc = result['uncertainty']
print(f'  Sequence entropy:    {unc["sequence_entropy"]:.4f}')
print(f'  Mean MI (epistemic): {unc["mean_mi"]:.4f}')
print(f'  Unique ratio:        {unc["unique_ratio"]:.2f}')
print(f'  Pairwise diversity:  {unc["pairwise_diversity"]:.3f}')
print()
print('=== Retrieval Uncertainty ===')
ret = result['retrieval_uncertainty']
print(f'  Mean posterior std:  {ret["mean_posterior_std"]:.4f}')
print(f'  Mean entropy:        {ret["mean_entropy"]:.4f}')

=== Generation Uncertainty ===
  Sequence entropy:    3.4752
  Mean MI (epistemic): 0.0439
  Unique ratio:        1.00
  Pairwise diversity:  0.839

=== Retrieval Uncertainty ===
  Mean posterior std:  0.1000
  Mean entropy:        0.4150


## 4. Interpretation

**What do these numbers mean?**

| Metric | Measures | High value means |
|--------|----------|------------------|
| Sequence entropy | Total uncertainty | Model is unsure overall |
| Mean MI | Epistemic (model) uncertainty | More data/training could help |
| Unique ratio | Sample diversity | Model produces varied answers |
| Pairwise diversity | Lexical variation | Answers differ in wording |
| Retrieval posterior std | Retrieval confidence spread | Retrieved docs may not be relevant |
| Retrieval entropy | Retrieval uncertainty | High = uncertain about doc relevance |

**Epistemic uncertainty (MI)** is the part we can reduce with better data or a bigger model.  
**Aleatoric uncertainty (expected entropy)** is irreducible noise — even a perfect model would be uncertain.

In a production system, you'd flag answers where MI is high as "the model isn't confident — consider human review."